## Bellman Optimality Equation solver
### MDP Formulation:
### $\mathcal{S} = \big\{(n_1, n_2): 0 \leq n_1, n_2 \leq M \big\}$
### $\mathcal{A} = [0,1]$
### Policy $\pi: \mathcal{S} \mapsto \mathcal{A}$ allocates the fraction of cores to type 1 jobs

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy.optimize import fsolve, minimize
from scipy.integrate import quad
from numpy import linspace, meshgrid, arange, empty, concatenate, newaxis, shape
from collections import deque

In [ ]:
def speed_up(amdahl_par, n_cores):
    speed_up = 1/(1 - amdahl_par*(1 - 1/max(1, n_cores)))
    return speed_up


# Returns transition probabilities from any state (n1, n2)
def transition_probabilities(model_pars, state, action):
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    
    prob_1 = lam*alpha if n1 < M else 0
    prob_2 = lam*(1-alpha) if n2 < M else 0
    prob_3 = min(n1, cores*action)*mu*speed_up(p1, cores*action/n1) if n1 > 0 else 0
    prob_4 = min(n2, cores*(1-action))*mu*speed_up(p2, cores*(1-action)/n2) if n2 > 0 else 0
    prob_5 = 1 - prob_1 - prob_2 - prob_3 - prob_4
    return [prob_1, prob_2, prob_3, prob_4, prob_5]


# Given current state, it samples the next state under the optimal core allocation policy
# which is evaluated by solving the Bellman optimality equation
def sample_next_state(model_pars, current_state, action):
    [n1, n2] = current_state
    
    possible_next_states = [[n1+1, n2], [n1, n2+1], [n1-1, n2], [n1, n2-1], [n1, n2]]
    indices = [0, 1, 2, 3, 4]
    probabilities = transition_probabilities(model_pars, current_state, action)
    next_state = np.array(possible_next_states[np.random.choice(indices, size = 1, p = probabilities)[0]])
    return next_state


# Following functions solve Bellman optimality equation
def A(model_pars, state, V):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    # Calculating A
    A_val = (n1 + n2) + \
            (lam*alpha*(V[n1+1, n2] - V[n1, n2]) if n1 < M else 0) + \
            (lam*(1-alpha)*(V[n1, n2+1] - V[n1, n2]) if n2 < M else 0)
    return A_val
    
    
def H_and_action(model_pars, state, V):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    
    # Evaluating best continuous action
    # min_cost = minimize(cost, 0.5 , method="L-BFGS-B", args = (model_pars, state, V), bounds = [(0,1)])
    # a_optimal = min_cost.x
    # H_optimal = V[n1, n2] + float(min_cost.fun)
    
    # Evaluating best discrete action from [0, 1/cores, 2/cores, ..., 1]
    costs_given_action = np.array([cost(i/cores, model_pars, state, V) for i in  range(cores+1)])
    index = np.argmin(costs_given_action)
    a_optimal = index/cores
    H_optimal = V[n1, n2] + costs_given_action[index]
    return H_optimal, a_optimal
    
    
def cost(action, model_pars, state, V):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    # Calculating cost
    cost = (min(n1, cores*action)*mu*speed_up(p1, cores*action/n1)*(V[n1-1, n2] - V[n1, n2]) if n1 > 0 else 0) + \
           (min(n2, cores*(1-action))*mu*speed_up(p2, cores*(1-action)/n2)*(V[n1, n2-1] - V[n1, n2]) if n2 > 0 else 0)
    return cost


def solve_MDP(model_pars):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    # We define old and new value function and update them.
    old_V = np.full((M+1, M+1), 1.0, dtype=np.float64)
    new_V = np.full((M+1, M+1), 1.0, dtype=np.float64)
    # Change in old_V and new_V should eventually converge to average cost per unit time.
    change_V = np.zeros((M+1, M+1))
    # Stores optimal action
    action = np.zeros((M+1, M+1))
    
    epsilon = 0.01 # Maximum permissible error
    delta = math.inf
    
    iteration = 1
    
    while delta >= epsilon:
        fraction_change = 1
        for i in range(M+1):
            for j in range(M+1):
                term_1 = A(model_pars, [i,j], old_V)
                term_2, a = H_and_action(model_pars, [i,j], old_V)
                action[i,j] = a
                new_V[i,j] = term_1 + float(term_2)
                fraction_change = max(fraction_change, (new_V[i,j] - old_V[i,j])/old_V[i,j] if old_V[i,j] > 0 else 1)
        change_V = new_V.copy() - old_V.copy()
        delta = change_V.max() - change_V.min()
        # print("Iteration = ", iteration, " Delta = ", delta)
        old_V = new_V.copy()
        iteration += 1
        
    return new_V, action


def bellman_optimal_policy(pars):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = pars
    # Scaling
    model_pars = scale_pars(pars)
    # Optimal value function and optimal policy
    V_optimal, pi_optimal  = solve_MDP(model_pars)
    relative_V_optimal = V_optimal - V_optimal[0,0]
    # Calculate relative Q values
    relative_Q_optimal = np.zeros((M+1, M+1, cores+1))
    for i in range(M+1):
        for j in range(M+1):
            for k in range(cores+1):
                probs = transition_probabilities(model_pars, [i,j], k/cores)
                future_cost = 0
                future_cost += probs[0]*relative_V_optimal[i+1, j] if i < M else 0
                future_cost += probs[1]*relative_V_optimal[i, j+1] if j < M else 0
                future_cost += probs[2]*relative_V_optimal[i-1, j] if i > 0 else 0
                future_cost += probs[3]*relative_V_optimal[i, j-1] if j > 0 else 0
                future_cost += probs[4]*relative_V_optimal[i, j]
                relative_Q_optimal[i, j, k] = (i+j) + future_cost
    
    return pi_optimal, relative_V_optimal, relative_Q_optimal


def scale_pars(pars):
    lam = pars[0]
    mu = pars[1]
    cores = pars[2]
    p1 = pars[3]
    p2 = pars[4]
    alpha = pars[5]
    M = pars[6]
    
    scaling_factor = lam + M*mu*(speed_up(p1, cores) + speed_up(p2, cores))
    scaled_pars = [lam/scaling_factor, mu/scaling_factor, cores, p1, p2, alpha, M]

    return scaled_pars

In [ ]:
# True parameters
lam = 2 # Arrival rate
mu = 1 # Mean service time
cores = 20 # Total number of cores
p1 = 0.4 # Bad jobs (p1 should be lower than p2)
p2 = 0.9 
alpha = 0.2
M = 10

true_pars = [lam, mu, cores, p1, p2, alpha, M]

bellman_pi_optimal, bellman_relative_V_optimal, bellman_relative_Q_optimal = bellman_optimal_policy(true_pars)

## Computing optimal actions given $\lambda, p_1, p_2, \alpha$

In [ ]:
# # Testing
# # print("Optimal actions are ")
# # print("")
# # print(np.matrix.round(bellman_pi_optimal,3))

# # Policy 1 heatmap
# fig, axes = plt.subplots(figsize=(18, 15))
# heatmap = sns.heatmap(bellman_pi_optimal, ax=axes, cmap="coolwarm", annot=False, cbar=True)

# # Move x-axis to top
# axes.xaxis.set_ticks_position('top')
# axes.xaxis.set_label_position('top')

# # Grid coordinate labels
# for i in range(bellman_pi_optimal.shape[0]):
#     for j in range(bellman_pi_optimal.shape[1]):
#         axes.text(j + 0.5, i + 0.5, f"{bellman_pi_optimal[i][j]}", ha='center', va='center', fontsize=10, color='black')

# heatmap.collections[0].colorbar.set_label("Fraction to $n_1$ (Bad Jobs)")
# axes.set_title("Heatmap of Bellman Optimal Policy", pad=30)
# axes.set_xlabel("$n_2$ (Good jobs)")
# axes.set_ylabel("$n_1$ (Bad Jobs)")
# plt.show()

## Visualizing Q-values

In [ ]:
# # Example usage for action k=0
# plot_q_heatmap(bellman_relative_Q_optimal, cores, action = 0)

In [ ]:
# # Example usage for action k=0
# plot_q_surface(bellman_relative_Q_optimal, cores, action = 1)

In [ ]:
# # Example usage for state (i=5, j=5)
# plot_q_vs_actions(bellman_relative_Q_optimal, state = [2, 3])